# 导入Pandas 包

In [1]:
import pandas as pd

# 预备知识

## 数据结构

每张牌2个维度：一个是数字，一个是花

花色我们用黑桃Spade，梅花Club，方块Diamonds，红桃Hearts用首字母标示S,C,D,H

数字用2，3，4，5，6，7，8，9，T，J，Q，K，A标示

用如下的方式标示手中的五张牌

In [106]:
a='9D 9H 9S 9C AC'
b='2C 7C 8C 9C TC'

c='2S 3S 4S 5S 7D'
d='2S 9S 7D 8S TD'

## python 的一些函数

In [107]:
#split函数，将字符串转化成list
a.split()

['9D', '9H', '9S', '9C', 'AC']

In [108]:
#list中for 语句的小技巧
print([i for i in a.split()])
print([a for a,b in a.split()])
print([b for a,b in a.split()])

['9D', '9H', '9S', '9C', 'AC']
['9', '9', '9', '9', 'A']
['D', 'H', 'S', 'C', 'C']


In [111]:
# 查找字符串中字符位置 index（）
print('--23456789TJQKA'.index('A'))
print('--23456789TJQKA'.index('7'))

14
7


In [115]:
# 语句判断
print(1==2)
print(2==2)
print(2>=2)
print('A'=='A')
print('AA'=='A')

False
True
True
True
False


In [120]:
#计算对象中，有多少个不重复的元素
print(set([1,2,2,3,4]))
print(set(['1','2','2','3','4']))
print(set('1222333'))
print(len(set([1,2,2,3,4])))

{1, 2, 3, 4}
{'3', '1', '4', '2'}
{'3', '1', '2'}
4


In [131]:
#计算对象中，某元素的个数
print([1,2,3,4,5].count(1))
print([1,1,1,2,3].count(1))

1
3


In [154]:
#比较多个对象大小max()按照顺序比较
print(max(9,8))
print(max([9,8,7,6,5],[10,9,8,7,6]))
print(max([10,8,7,6,5],[10,9,8,7,6]))
print(max([10,9,7,6,5],[10,9,8,7,6]))
print(max((9,[9,8,7,6,5]),(8,[10,8,7,6,5])))

9
[10, 9, 8, 7, 6]
[10, 9, 8, 7, 6]
[10, 9, 8, 7, 6]
(9, [9, 8, 7, 6, 5])


# 德扑判断卡牌大小

## 按照数字对五张牌进行排序

In [2]:
def card_ranks(hand):
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return  [5, 4, 3, 2, 1] if ranks == [14, 5, 4, 3, 2] else ranks

In [3]:
a='9D 9H 9S 9C AC'
print(card_ranks(a.split()))

[14, 9, 9, 9, 9]


## 判断顺子/同花/两对/n张相同牌

In [4]:
#ranks 指的是数字排序后的list，例如[5，4，3，2，1]
#hand 指的是牌，例如['1S','2S','6S','8S','9S']

# 判断顺子 
def straight(ranks):
    return (max(ranks) - min(ranks)) == 4 and len(set(ranks)) == 5 #最大牌减最小牌等于4，且五张牌不重复

#判断同花
def flush(hand):
    suit = [s for r, s in hand]
    return len(set(suit)) == 1 #五张牌的花色相同

#判断n张牌相同
def kind(n, ranks):
    for s in ranks:
        if ranks.count(s) == n : return s #如果某元素在对象中的个数等于我们定义的n，则返回该元素具体数值
    return None

#判断是否有两对
def two_pair(ranks):
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks))) #在有两对儿的情况下，因为赋新值会使得原值被覆盖，所以把原来的rank换个方向，就会得到另一对儿
    if pair != lowpair:
        return (pair, lowpair)
    else:
        return None

In [5]:
ranks_test=[9,8,7,5,5]
hand_test=['1S','2S','6S','8S','9S']

In [6]:
straight(ranks_test)

False

In [7]:
flush(hand_test)

True

In [8]:
kind(2,ranks_test)

5

In [9]:
two_pair(ranks_test) #没有输出，是因为没有两对

## 为不同牌型赋予权重

In [10]:
def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand) 
    if straight(ranks) and flush(hand):                #同花顺
        return (9, max(ranks))                         #返回（权重9，顺子的最大牌面）
    elif kind(4, ranks):                               #炸弹
        return (8, kind(4, ranks), kind(1, ranks))     #返回（权重8，炸弹的牌面，剩余的一张牌的大小）
    elif kind(3, ranks) and kind(2, ranks):            #3带2
        return (7, kind(3, ranks), kind(2, ranks))     #返回（权重7，三张的牌面，两张的牌面）
    elif flush(hand):                                  #同花
        return (6,ranks)                               #返回（权重6，五张牌的从大到小的排序）
    elif straight(ranks):                              #顺子
        return (5, max(ranks))                         #返回（权重5，顺子的最大牌面）
    elif kind(3, ranks):                               #三张
        return (4, kind(3, ranks), ranks)              #返回（权重4，三张的牌面，五张牌的从大到小的排序）
    elif two_pair(ranks):                              #两对
        return (3, two_pair(ranks), ranks)             #返回（权重3，（较大的一对儿的牌面，较小的一对儿的牌面），五张牌的从大到小的顺序）
    elif kind(2, ranks):                               #两张
        return (2, kind(2, ranks), ranks)              #返回（权重2，两张的牌面，五张牌的从大到小的顺序）
    else:                                              #一张
        return (1, ranks)                              #返回（权重1，五张牌的从大到小的顺序）

## 比较大小

In [11]:
def poker(hands):
    return max(hands, key=hand_rank) # 利用max（）是按照顺序比较，所以首先比较权重，然后如果权重一样，则比较下一位

In [ ]:
for i in []

In [84]:
def whowins(hands):
    return [hands.index(i) for i in hands if hand_rank(i)==hand_rank(poker(hands))]

# Test

In [73]:
a='9D 9H 9S 9C AC'.split()
b='2C 7C 8C 9C TC'.split()

c='2S 9S 7D 8C TC'.split()
d='2S 9S 7D 8S TH'.split()
e='2S 9S 7D 8H TS'.split()
f='2S 9S 7D 8D TD'.split()

In [87]:
hands=[c,d,e,f]

In [88]:
whowins(hands)

[0, 1, 2, 3]

In [83]:
[hands.index(i) for i in hands if hand_rank(i)==hand_rank(poker(hands))]

[0, 1, 2, 3]

In [23]:
poker([a,b,c,d])

['9D', '9H', '9S', '9C', 'AC']

In [25]:
poker([d,c])

['2S', '9S', '7D', '8S', 'TD']

In [27]:
[a,b,c,d].index(poker([c,d]))

2

In [28]:
len([a,b,c,d])

4

In [66]:
def whowins(hands):
    loc=hands.index(poker(hands))
    whowin=[]
    for x in hands:
        mid=list(hands)
        mid.insert(0,x)
        mid_loc=mid.index(poker(mid))
        if mid_loc!=(loc+1):
            whowin.append(hands.index(x))
    return whowin

In [74]:
hands=[c,d,e,f]

In [44]:
hands.index(poker(hands))

0

In [48]:
mid=hands.copy

In [50]:
mid=list(hands)

In [56]:
mid.insert(0,a)

In [55]:
hands

[['9D', '9H', '9S', '9C', 'AC'],
 ['2C', '7C', '8C', '9C', 'TC'],
 ['2S', '9S', '7D', '8C', 'TD'],
 ['2S', '9S', '7D', '8S', 'TD']]

In [47]:
print(mid)

None


In [57]:
mid

[['9D', '9H', '9S', '9C', 'AC'],
 ['9D', '9H', '9S', '9C', 'AC'],
 ['9D', '9H', '9S', '9C', 'AC'],
 ['2C', '7C', '8C', '9C', 'TC'],
 ['2S', '9S', '7D', '8C', 'TD'],
 ['2S', '9S', '7D', '8S', 'TD']]

In [40]:
hands.index(poker(hands))

0

In [75]:
whowins(hands)

[0, 1, 2, 3]

In [76]:
hands[0]

['2S', '9S', '7D', '8C', 'TC']